In [28]:
import pandas as pd

# users = pd.read_csv('data/users_export.csv')
ratings = pd.read_csv('data/ratings_export.csv')
movies = pd.read_csv('data/movie_data.csv', on_bad_lines='skip')
my_ratings = pd.read_csv('data/processed/ratings_tmdb_cleaned.csv')
watched = pd.read_csv('data/watched.csv')

# Merge datasets on movie_id to create a comprehensive dataset
user_ratings = pd.merge(ratings, movies, left_on='movie_id', right_on='movie_id')

In [ ]:
user_ratings.head()

,_id_x,movie_id,rating_val,user_id,_id_y,genres,image_url,imdb_id,imdb_link,movie_title,...,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
1,5fc57ca06758f69634538bad,feast-2014,10,ingridgoeswest,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
2,5fc57cbd6758f696345475a3,feast-2014,8,dirkh,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
3,5fc57ce06758f6963455400b,feast-2014,10,childrenofmen,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
4,5fc57cf36758f69634558b0e,feast-2014,8,suspirliam,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0


In [ ]:
len(user_ratings)

11079666

In [ ]:
len(my_ratings)

277

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'tmdb_id' in movies DataFrame and 'id' in ratings DataFrame are the columns to be used
# The 'left_on' and 'right_on' parameters in merge function need to be updated accordingly

# Updated merge to use 'tmdb_id' from movies DataFrame
merged = my_ratings.merge(movies[['tmdb_id', 'movie_title', 'year_released']], 
                          left_on='id',  # Update this based on the actual column in my_ratings that matches tmdb_id in movies
                          right_on='tmdb_id', 
                          how='inner')

# remove ratings of 0 (not rated)
merged = merged[merged['Rating'] != 0]

my_ratings_updated = merged[['tmdb_id', 'Rating']].copy()  # Use .copy() to explicitly create a copy

my_ratings_updated = my_ratings_updated.groupby('tmdb_id').agg({'Rating': 'mean'}).reset_index()
my_ratings_updated['user_id'] = "brimell"  # Assign 'user_id' = 0 directly

my_ratings_updated, len(my_ratings_updated)


(     tmdb_id  Rating  user_id
 0       85.0     3.5  brimell
 1       89.0     3.5  brimell
 2      105.0     4.0  brimell
 3      137.0     4.0  brimell
 4      155.0     4.5  brimell
 ..       ...     ...      ...
 95  616037.0     3.0  brimell
 96  673593.0     2.5  brimell
 97  693134.0     4.5  brimell
 98  695721.0     3.5  brimell
 99  696806.0     4.0  brimell
 
 [100 rows x 3 columns],
 100)

In [ ]:
# Map movie IDs and user IDs to indices for creating a sparse matrix
tmdb_id_to_idx = {tmdb_id: i for i, tmdb_id in enumerate(user_ratings['tmdb_id'].unique())}
user_id_to_idx = {user_id: i + 1 for i, user_id in enumerate(user_ratings['user_id'].unique())} # add one to avoid 0 index so that we can use 0 for ourselves

# Add your ratings to the user ratings DataFrame
combined_ratings = pd.concat([user_ratings, my_ratings_updated.rename(columns={'Rating': 'rating_val'})])
# remove empty rows
combined_ratings = combined_ratings.dropna(subset=['user_id', 'rating_val'])

user_id_to_idx["brimell"] = 0  # Add ourselves to the mapping
user_id_to_idx

{'deathproof': 1,
 'ingridgoeswest': 2,
 'dirkh': 3,
 'childrenofmen': 4,
 'suspirliam': 5,
 'nycsubwayrat': 6,
 'davidfinchher': 7,
 'screeningnotes': 8,
 'blockbustedpod': 9,
 'uncutgems': 10,
 'oliviawildehawt': 11,
 'drivefiction19': 12,
 'willhunting': 13,
 'jvince': 14,
 'bethanywilson': 15,
 'mariamicia13': 16,
 'ucrazyutaraptor': 17,
 'haybatman': 18,
 'artpig': 19,
 'lise': 20,
 'shanderson88': 21,
 'kevinyang': 22,
 'cathyk': 23,
 'jimmyvon789': 24,
 'letmeexplain': 25,
 'dahans': 26,
 'yorel1976': 27,
 'bbeksinski': 28,
 'nickmcc2': 29,
 'rsolsjo': 30,
 'therealmattc': 31,
 'farriercollins': 32,
 'mortom': 33,
 'cvall96': 34,
 'hstrawberry': 35,
 'jameshealey': 36,
 'eclectic_em': 37,
 'balanders': 38,
 'griffinstenzel': 39,
 'mattybsaps': 40,
 'moviefan999': 41,
 'anhonestmess': 42,
 'jakeperalta': 43,
 'peacefulstoner': 44,
 'adrianam': 45,
 'crackfox45': 46,
 'cgrunyon': 47,
 'axelhagne': 48,
 'mank': 49,
 'annakendrick': 50,
 'theccritic': 51,
 'tarantinole': 52,
 'ashon

In [ ]:
# Filter combined_ratings for movies rated 4.5 or 5.0
# high_rated_combined_ratings = combined_ratings[combined_ratings['rating_val'].isin([4.5, 5.0])]
high_rated_combined_ratings = combined_ratings[combined_ratings['rating_val']]

# Create a sparse matrix of ratings
rows = combined_ratings['user_id'].map(user_id_to_idx)
cols = combined_ratings['tmdb_id'].map(tmdb_id_to_idx)
data = combined_ratings['rating_val']
ratings_matrix = csr_matrix((data, (rows, cols)), shape=(len(user_id_to_idx), len(tmdb_id_to_idx)))

# Compute cosine similarity between users
user_similarity = cosine_similarity(ratings_matrix)
user_similarity

array([[1.        , 0.12588118, 0.1426512 , ..., 0.06880356, 0.        ,
        0.        ],
       [0.12588118, 1.        , 0.50547582, ..., 0.02844054, 0.01522636,
        0.02131691],
       [0.1426512 , 0.50547582, 1.        , ..., 0.02127927, 0.        ,
        0.        ],
       ...,
       [0.06880356, 0.02844054, 0.02127927, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.01522636, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.02131691, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
# Since you are user_id 0, find the most similar users to you
top_100_similar_users_indices = np.argsort(-user_similarity[0])[1:1000]  # Get indices of top x similar users

# print the usernames of the top 100 similar users and how similar they are to you
for idx in top_100_similar_users_indices:
    print(f"User ID: {combined_ratings['user_id'].unique()[idx]}, Similarity: {user_similarity[0][idx]}")


User ID: deathproof, Similarity: 0.9999999999999996
User ID: shruglord, Similarity: 0.3049952053681799
User ID: ollidigital, Similarity: 0.30477994961482185
User ID: sambarboza_, Similarity: 0.30296623975078146
User ID: scravvny, Similarity: 0.28801033100755763
User ID: viki22, Similarity: 0.2865860132269306
User ID: simon__, Similarity: 0.2864950952325238
User ID: coopsido, Similarity: 0.2838859039245743
User ID: jackandjill, Similarity: 0.2822740733914849
User ID: erfrost, Similarity: 0.27743336474842945
User ID: soundbyter, Similarity: 0.2768527327772321
User ID: hoppernick27, Similarity: 0.27281410415072144
User ID: suryanaga, Similarity: 0.27225031154290585
User ID: mrrobertman, Similarity: 0.2711074617144814
User ID: mrgg_06, Similarity: 0.26963360033884426
User ID: moviec, Similarity: 0.26805972594466476
User ID: iwannabehoo, Similarity: 0.2665604363748806
User ID: james_cody2020, Similarity: 0.2661902024895151
User ID: awi5132, Similarity: 0.265686310646846
User ID: sydneyb1233

KeyboardInterrupt: 

In [ ]:
my_rated_movies = set(my_ratings_updated['tmdb_id'])

# Initialize a dictionary to hold recommended movies and the users who recommended them
recommended_movies_details = {}

for user_index in top_100_similar_users_indices:
    user_id = list(user_id_to_idx.keys())[list(user_id_to_idx.values()).index(user_index)]
    
    high_rated_movies_by_user = combined_ratings[(combined_ratings['user_id'] == user_id)]
    
    for _, row in high_rated_movies_by_user.iterrows():
        tmdb_id = row['tmdb_id']
        if tmdb_id not in my_rated_movies:
            if tmdb_id not in recommended_movies_details:
                recommended_movies_details[tmdb_id] = {'users': [user_id], 'ratings': [row['rating_val']]}
            else:
                recommended_movies_details[tmdb_id]['users'].append(user_id)
                recommended_movies_details[tmdb_id]['ratings'].append(row['rating_val'])

# Limit to top [whatever] based on the number of users recommending the movie
recommended_movies_ids = sorted(recommended_movies_details, key=lambda x: len(recommended_movies_details[x]['users']), reverse=True)[:100]

# Fetch movie titles and stats
recommended_titles_and_stats = []
for tmdb_id in recommended_movies_ids:
    movie_title = movies[movies['tmdb_id'] == tmdb_id]['movie_title'].iloc[0]
    avg_rating = np.mean(recommended_movies_details[tmdb_id]['ratings'])
    num_users = len(recommended_movies_details[tmdb_id]['users'])
    recommended_titles_and_stats.append({
        'title': movie_title,
        'average_rating': avg_rating,
        'recommended_by_users_count': num_users,
        'recommended_by_user_ids': recommended_movies_details[tmdb_id]['users']
    })

# sort by average rating
recommended_titles_and_stats = sorted(recommended_titles_and_stats, key=lambda x: x['average_rating'], reverse=True)

# Display recommended movies along with stats
for movie in recommended_titles_and_stats:
    print(f"Title: {movie['title']}, Avg Rating: {movie['average_rating']:.2f}, Recommended by {movie['recommended_by_users_count']} Users, User IDs: {movie['recommended_by_user_ids']}")

Title: Parasite, Avg Rating: 9.44, Recommended by 78 Users, User IDs: ['sdist', 'ecolei', 'woozy_', 'luckyspecialist', 'duz111', 'canasian', 'gtsad', 'nugg4life', 'leeyummie', 'robster8', 'currycing', 'leoningall', 'carriefornia', 'wackydawg', 'paristsambo', 'trennison1', 'samdoesntmiss', 'jb97', 'bananablitzer1', 'sophiaplanos', 'sealinfinnish', 'pog_monky', 'jcarney2017', 'armaandave', 'hjshrimpton', 'mbillcliff', 'undog', 'rented', 'lukreviews', 'cesourius', 'awade', 'fzilon', 'hughmcintosh207', 'greggoursaud', 'gabbvautour', 'edwilled', 'andrew_007', 'jaxlovesmovies', 'younghannibal', 'squidwardo', 'darklord98', 'madagaw', 'gxenick', 'leomarvindiaz', 'jakobjonathan', 'henrikr', 'kongstrong', 'hockeyfan15', 'jonahd', 'h3nry', 'quinnkong', 'jwill904', 'kvashaihor', 'hedwiggggg', 'kebabvchlebu', 'ntgntg3', 'cosmicpaint', 'felix__v', 'timothywang', 'tareqzaghal', 'mrwan', 'wayne78', 'ulrikirgens', 's_o_n_n_y', 'moviemicah', 'hootington', 'jcpeters10', 'lgoree', 'willyum_', 'lukesopinio

In [ ]:
# output to file

with open('data/recommended_movies.txt', 'w') as f:
    for movie in recommended_titles_and_stats:
        f.write(f"Title: {movie['title']}, Avg Rating: {movie['average_rating']:.2f}, Recommended by {movie['recommended_by_users_count']} Users, User IDs: {movie['recommended_by_user_ids']}\n")